In [10]:
import imutils
import cv2
import numpy as np

def detectLetters(image,kernelSize,rtvMode,approx):
    boundRect = [None]
    #image = cv2.imread(inputFilename)
    
 
    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
 
    # show the original image and the edge detected image
    cv2.imwrite('originals/dltme1.png',edged)
    print ("STEP 1: Edge Detection")

    #Binary Dialation
    kernelSize = (17,3)
    element = cv2.getStructuringElement (cv2.MORPH_RECT, kernelSize )
    img_threshold = cv2.morphologyEx(edged,cv2.MORPH_CLOSE,element)
    cv2.imwrite('originals/dltme2.png',img_threshold)
    print('Binary dialation done...')
    
    #Find Contours
    im2,contours,hierarchy = cv2.findContours(img_threshold,rtvMode,approx)
    contours_poly = [None]*len(contours)
    for i in range(0,len(contours)):
        if len(contours[i])>100:            
            contours_poly[i]=(cv2.approxPolyDP(contours[i], 3, True))            
            x,y,w,h = cv2.boundingRect(contours_poly[i])            
            if w>h:
                temp = x,y,w,h
                boundRect.append(temp)
    return boundRect

def convertToBW(image):
    gray_masked = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    (thresh, im_bw) = cv2.threshold(gray_masked, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return im_bw

def setImage(image):   
    #print('Before Resizing = ',image.shape)
    (height,width) = image.shape   
    if height > 20:    
        if width < 20 :
            #crop
            image = image[1:30,:]
        else:            
            #print(image.shape)
            image = imutils.resize(image,height=20)
    (height,width) = image.shape
    if width > 20:
        if height < 10:
            #Crop
            image = image[:,1:30]
        else:
            image = imutils.resize(image,width=20)           
        
    #print('After Resizing = ',image.shape)
    #image = imutils.resize(image, width=20)
    #Draw a white canvas
    white_img = np.zeros((40,40), np.uint8)
    white_img[:,:] = (255)
    #Paste the image onto the white canvas
    x_offset=y_offset=1    
    white_img[y_offset:y_offset+image.shape[0], x_offset:x_offset+image.shape[1]] = image
    return white_img

def extractTextArea(inputFile,kernelSize,rtvMode,approx):
    j=0
    #implementation    
    img = cv2.imread(inputFile)
    ratio = img.shape[0] / 500.0
    orig = img.copy()
    img = imutils.resize(img, height = 500)
    letterBBoxes=detectLetters(img,kernelSize,rtvMode,approx)
    textArea_Count = 0
    for i in range(0,len(letterBBoxes)-1):
        if letterBBoxes[i] != None:   
            textArea_Count = textArea_Count + 1
            temp = letterBBoxes[i]
            cv2.rectangle(orig,(temp[0],temp[1]),(temp[0]+temp[2],temp[1]+temp[3]),(0,255,0),3)
            crop_image = img[temp[1]:temp[1]+temp[3], temp[0]:temp[0]+temp[2]]
            cv2.imwrite('dltme/textArea'+str(textArea_Count)+'.png',crop_image)
            bw_img = convertToBW(crop_image)
           
            (height1,width1) = bw_img.shape
            init_column = 0
            end_column = 5
            i=0
            char_list = []
            
            
            while init_column + end_column + i < width1:    
                sumValue = bw_img[:,init_column+end_column+i].sum()                
                if (sumValue / 255) == height1: 
                    
                    char_image = bw_img[:,init_column:init_column+end_column+i]
                    init_column = init_column + end_column+i+1
                    i=0                    
                    #j=j+1
                    char_list.append(char_image)                
                i=i+1
            if (i>5):
                char_image = bw_img[:,init_column:init_column+end_column+i-1]
                char_list.append(char_image) 
            
            print(len(char_list))
            for entry in char_list:
                #temp_img = cv2.blur(setImage(entry),(2,2))
                temp_img = cv2.blur(setImage(entry),(2,2))
                cv2.imwrite('dltme/dltme_'+str(j)+'.png',temp_img);j=j+1                
                prediction = loaded_model.predict(temp_img.reshape(1,1600))
                print(str(prediction))
                #image = cv2.blur(image,(2,2))
                


In [20]:
import pickle
loaded_model = pickle.load(open('Model output/AlphabetRecognition.sav', 'rb'))
kernelSize = (17,3)

#extractTextArea('originals/Pancard.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
extractTextArea('originals/Pancard_Web_1_1.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

STEP 1: Edge Detection
Binary dialation done...
7
[b'K']
[b'N']
[b'K']
[b'K']
[b'I']
[b'K']
[b'N']
6
[b'7']
[b'1']
[b'1']
[b'1']
[b'1']
[b'1']
10
[b'H']
[b'N']
[b'Z']
[b'P']
[b'M']
[b'Z']
[b'5']
[b'D']
[b'1']
[b'F']
22
[b'1']
[b'K']
[b'I']
[b'N']
[b'K']
[b'1']
[b'K']
[b'1']
[b'I']
[b'T']
[b'I']
[b'1']
[b'7']
[b'K']
[b'K']
[b'I']
[b'1']
[b'K']
[b'N']
[b'K']
[b'K']
[b'I']
3
[b'1']
[b'1']
[b'1']
1
[b'1']
1
[b'M']
2
[b'E']
[b'1']
1
[b'1']
9
[b'1']
[b'A']
[b'I']
[b'7']
[b'F']
[b'1']
[b'T']
[b'P']
[b'P']
9
[b'D']
[b'U']
[b'R']
[b'A']
[b'I']
[b'S']
[b'A']
[b'M']
[b'T']
12
[b'G']
[b'1']
[b'M']
[b'A']
[b'N']
[b'I']
[b'K']
[b'A']
[b'N']
[b'G']
[b'A']
[b'N']
1
[b'O']
12
[b'I']
[b'N']
[b'I']
[b'D']
[b'M']
[b'E']
[b'1']
[b'1']
[b'E']
[b'1']
[b'E']
[b'N']
9
[b'D']
[b'1']
[b'1']
[b'O']
[b'7']
[b'T']
[b'N']
[b'D']
[b'U']
1
[b'1']
1
[b'1']
1
[b'1']
1
[b'1']
5
[b'1']
[b'1']
[b'1']
[b'1']
[b'1']
